- https://github.com/tensorflow/models/tree/master/research/im2txt
- https://yashk2810.github.io/
- https://github.com/yashk2810/Image-Captioning/blob/master/Image%20Captioning%20InceptionV3.ipynb
- https://arxiv.org/pdf/1609.06647.pdf

In [1]:
from PIL import Image
from glob import glob
import os
import re
import pdb
import pickle

In [2]:
dpath = '/data/image_captioning/flikr8/'
img_path = dpath+'Flicker8k_Dataset/'


- 8091 images
- 5 captions for each image.
- maybe just take the first caption to start
- 6000 train images, 1000 dev images, 1000 test images
- will need to separate into different directories.
- will probably want dataset captions in different files.
- all images seem to be 500 tall
- https://yashk2810.github.io/
- https://github.com/yashk2810/Image-Captioning/blob/master/Image%20Captioning%20InceptionV3.ipynb
- now you have captions. You'll need to tokenize them and embed them.
- the next step would probably be to get the activations for the images
- if you want to precompute them you'll need to save them in some type of specified order so you can match them with the captions.
- 

In [12]:
imgs = glob(img_path+'/*.jpg'); len(imgs)

8091

In [21]:
Image.open(imgs[127]).size

(500, 387)

## Caption Processing

First, maybe just take the first caption for each image.
- construct a regex that matches the first captions

### Create file of only first captions

In [1]:
match = '1003163366_44323f5815.jpg#0 '
non_match = '1003163366_44323f5815.jpg#1 '

In [33]:
first_cap_re = re.compile('^.+\.jpg#0')

In [7]:
first_cap.match(match)

<_sre.SRE_Match object; span=(0, 27), match='1003163366_44323f5815.jpg#0'>

In [8]:
first_cap.match(non_match)

In [36]:
all_caps_path = dpath+'Flickr8k.token.txt'
first_caps = []
for line in open(all_caps_path):
    if first_cap_re.match(line):
        line_arr = line.split('\t')
        img_nm = line_arr[0][:-2]
        cap = line_arr[1].strip()
        first_caps.append((img_nm, cap))

In [37]:
len(first_caps)

8091

In [47]:
first_caps[0]

('1000268201_693b08cb0e.jpg',
 'A child in a pink dress is climbing up a set of stairs in an entry way .')

In [52]:
pickle.dump?

In [54]:
pickle.dump(first_caps, open(dpath+'all_first_caps.pkl', 'wb'))

In [57]:
first_caps = pickle.load(open(dpath+'all_first_caps.pkl', 'rb')); first_caps[0]

('1000268201_693b08cb0e.jpg',
 'A child in a pink dress is climbing up a set of stairs in an entry way .')

### Separate train, dev and test captions

In [70]:
def getImgCapForSet(set_names_path, all_imgs_caps):
    nms = [line.strip() for line in open(set_names_path)]
    return [img_cap for img_cap in all_imgs_caps if img_cap[0] in nms]       

In [71]:
tr_nms_caps = getImgCapForSet(dpath+'Flickr_8k.trainImages.txt', first_caps)

In [72]:
test_nms_caps = getImgCapForSet(dpath+'Flickr_8k.testImages.txt', first_caps)

In [73]:
dev_nms_caps = getImgCapForSet(dpath+'Flickr_8k.devImages.txt', first_caps)

In [74]:
print(len(tr_nms_caps)); tr_nms_caps[0]

6000


('1000268201_693b08cb0e.jpg',
 'A child in a pink dress is climbing up a set of stairs in an entry way .')

In [75]:
print(len(test_nms_caps)); test_nms_caps[0]

1000


('1056338697_4f7d7ce270.jpg',
 'A blond woman in a blue shirt appears to wait for a ride .')

In [76]:
print(len(dev_nms_caps)); dev_nms_caps[0]

1000


('1022454332_6af2c1449a.jpg',
 'A child and a woman are at waters edge in a big city .')

In [78]:
pickle.dump(tr_nms_caps, open(dpath+'train_first_caps.pkl', 'wb'))
pickle.dump(test_nms_caps, open(dpath+'test_first_caps.pkl', 'wb'))
pickle.dump(dev_nms_caps, open(dpath+'dev_first_caps.pkl', 'wb'))

### Move Images

In [5]:
tr_img_names = [nm.strip() for nm in open(dpath+ 'Flickr_8k.trainImages.txt')]

tr_img_path = img_path+'train'
for nm in tr_img_names:
    os.rename(os.path.join(img_path, nm), os.path.join(tr_img_path, nm))

In [20]:
def moveImgs(nm_path, out_path, img_path=img_path):
    img_names = [nm.strip() for nm in open(nm_path)]
    for nm in img_names:
        os.rename(os.path.join(img_path, nm), os.path.join(out_path, nm))

In [21]:
moveImgs(dpath+ 'Flickr_8k.devImages.txt', img_path+'dev')

In [24]:
moveImgs(dpath+ 'Flickr_8k.testImages.txt', img_path+'test')

### Create Sample

In [10]:
from shutil import copy

In [11]:
samp_path =img_path+'samp/wrap/'

In [6]:
tr_imgs = glob(img_path+'train/wrap/*.jpg')

In [8]:
first4 = tr_imgs[:4]

In [9]:
first4

['/data/image_captioning/flikr8/Flicker8k_Dataset/train/wrap/1082252566_8c79beef93.jpg',
 '/data/image_captioning/flikr8/Flicker8k_Dataset/train/wrap/2888408966_376c195b3f.jpg',
 '/data/image_captioning/flikr8/Flicker8k_Dataset/train/wrap/2469620360_6c620c6f35.jpg',
 '/data/image_captioning/flikr8/Flicker8k_Dataset/train/wrap/2181117039_c4eea8036e.jpg']

In [12]:
for f in first4:
    copy(f, os.path.join(samp_path, os.path.basename(f)))

In [13]:
len(glob(samp_path+'/*.jpg'))

4